# License

    Jupyter notebook for demonstrating testing in Python
    Copyright (C) 2018 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import pytest
from ipytest import run_pytest, clean_tests

# Documentation for pytest
https://docs.pytest.org/en/latest/reference.html

In [2]:
#Import packages we need
import numpy as np
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray
import pycuda.driver as cuda_driver

import IPythonMagic

In [3]:
%cuda_context_handler context

In [4]:
cuda_kernel = """
__global__ void addKernel(float* c, float* a, float* b) {
    unsigned int i = blockIdx.x*blockDim.x + threadIdx.x;
    c[i] = a[i] + b[i];
}
"""
module = cuda_compiler.SourceModule(cuda_kernel)
add_kernel = module.get_function("addKernel");


def gpuAdd(a, b):
    c = np.empty_like(a)

    #Upload data to the device
    a_g = GPUArray(a.shape, np.float32)
    b_g = GPUArray(b.shape, np.float32)
    a_g.set(a)
    b_g.set(b)

    #Allocate output data
    c_g = GPUArray(c.shape, np.float32)

    #Execute program on device
    add_kernel(c_g, a_g, b_g, block=(a.shape[0], 1, 1), grid=(1,1))

    #Copy data from device to host
    c_g.get(c)
    
    return c

In [5]:
clean_tests()

def test_constants():
    assert True == True
    assert 4 == 4
    assert 5 == 4

def test_vector_add_gpu():
    n = 50
    
    #Create test input / output data
    a = np.random.rand(n).astype(np.float32)
    b = np.random.rand(n).astype(np.float32)
    assert gpuAdd(a, b) == pytest.approx(a + b)
    
    a = np.ones(n).astype(np.float32)
    b = np.ones(n).astype(np.float32)
    assert gpuAdd(a, b) == pytest.approx(2*np.ones(n))
    

run_pytest(filename='05 PyTest.ipynb', pytest_options=['-vvv'])

============================= test session starts ==============================
platform linux -- Python 3.6.6, pytest-3.8.2, py-1.6.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /home/ubuntu/jupyter_notebooks/Andre Brodtkorb/MilanoGPU2018/notebooks, inifile:
collecting ... collected 2 items

05 PyTest.py::test_constants <- <ipython-input-5-5e94da710372> FAILED    [ 50%]
05 PyTest.py::test_vector_add_gpu <- <ipython-input-5-5e94da710372> PASSED [100%]

=================================== FAILURES ===================================
________________________________ test_constants ________________________________

    def test_constants():
        assert True == True
        assert 4 == 4
>       assert 5 == 4
E       AssertionError

<ipython-input-5-5e94da710372>:6: AssertionError
====================== 1 failed, 1 passed in 0.06 seconds ======================


1